In [1]:
import findspark
findspark.init()

In [10]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

In [4]:
sc = SparkSession.builder.appName("updateSpecificRecords").config("master", "local").getOrCreate()

25/11/03 21:36:07 WARN Utils: Your hostname, shasankperiwal resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlp0s20f3)
25/11/03 21:36:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/03 21:36:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
df = sc.read.csv("Spark/assets/updateCity.csv", header=True, inferSchema=True)
df.show(5, truncate = False)

+--------+------+----------+-------------+----+
|username|userid|phno      |company      |city|
+--------+------+----------+-------------+----+
|user_001|U1001 |9300522056|Tech Mahindra|DEL |
|user_002|U1002 |9790483348|Deloitte     |GAU |
|user_003|U1003 |9238354411|Capgemini    |BLR |
|user_004|U1004 |9795852478|Infosys Ltd  |HYD |
|user_005|U1005 |9299019722|Wipro        |HYD |
+--------+------+----------+-------------+----+
only showing top 5 rows



In [11]:
blr_df = df.filter(col("city") == 'BLR').withColumn("rnk", row_number().over(Window.orderBy(col("userid")))).filter(col("rnk")<=2).select(col("userid"))
blr_df.show()

25/11/03 21:43:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/03 21:43:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/03 21:43:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/03 21:43:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/03 21:43:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+
|userid|
+------+
| U1003|
| U1013|
+------+



In [14]:
final_df = df.alias("a").join(blr_df.withColumn("mrk", lit(1)).alias("b"), on = "userid", how = "left")\
            .withColumn("city", when(col("mrk") == 1, "KOL").otherwise(col("city"))).drop("mrk")
final_df.show()

25/11/03 23:04:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/03 23:04:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/03 23:04:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/03 23:04:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+--------+----------+-------------+----+
|userid|username|      phno|      company|city|
+------+--------+----------+-------------+----+
| U1001|user_001|9300522056|Tech Mahindra| DEL|
| U1002|user_002|9790483348|     Deloitte| GAU|
| U1003|user_003|9238354411|    Capgemini| KOL|
| U1004|user_004|9795852478|  Infosys Ltd| HYD|
| U1005|user_005|9299019722|        Wipro| HYD|
| U1006|user_006|9331810831|    Accenture| HYD|
| U1007|user_007|9246342542|    Accenture| HYD|
| U1008|user_008|9803615422|     Deloitte| HYD|
| U1009|user_009|9654302352|          TCS| DEL|
| U1010|user_010|9695930644|    Cognizant| HYD|
| U1011|user_011|9696930811|     Deloitte| GAU|
| U1012|user_012|9979932888|    Capgemini| DEL|
| U1013|user_013|9619905582|     EY India| KOL|
| U1014|user_014|9334318874|    IBM India| DEL|
| U1015|user_015|9718570144|  Infosys Ltd| HYD|
| U1016|user_016|9112510070|     EY India| DEL|
| U1017|user_017|9331902097|    Cognizant| HYD|
| U1018|user_018|9287748252|          TC